# data preproccessing


## import modules and configure notebook

In [88]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
pd.set_option('max.rows', None)
pd.set_option('max.columns', None)


from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
%matplotlib inline

### configurations
* data_input_path -> string, filepath to data to be read in

* grouping -> boolean, if set to True them many sites are grouped
* reduced_grouping -> boolean, if set to True then less sites grouped, only bedrock sites 'WB' and 'BX are grouped into one class and superficial sites 'SV' and 'SE' are grouped into one class.
* raw -> boolean, if set to True then no grouping done
* drop_semi_bedrock ->  True|False, if set to True then some bedrock sites deemed to be semi-bedrock sites are not used for classification



In [89]:
data_input_path = '../data/raw_data.csv'


grouping = False

grouping_reduced = True
raw = False
drop_semi_bedrock = False


### Import data

In [47]:
my_data = pd.read_csv(data_input_path)

In [48]:
my_data.Geology.value_counts()

Bedrock        808
Superficial    435
Artefacts      363
Name: Geology, dtype: int64

### features

In [49]:
my_data.columns.values[9:-1]

array(['Li7', 'Be9', 'B11', 'Mg24', 'Al27', 'Si28', 'P31', 'S33', 'K39',
       'Ca42', 'Sc45', 'Ti47', 'V51', 'Cr52', 'Mn55', 'Fe56', 'Co59',
       'Ni60', 'Cu63', 'Zn68', 'Ga69', 'Ge72', 'As75', 'Rb85', 'Sr88',
       'Y89', 'Zr90', 'Nb93', 'Mo95', 'Cd111', 'In115', 'Sn118', 'Cs133',
       'Ba137', 'La139', 'Ce140', 'Pr141', 'Nd146', 'Sm147', 'Eu153',
       'Gd157', 'Tb159', 'Dy163', 'Ho165', 'Er166', 'Tm169', 'Yb172',
       'Lu175', 'Hf178', 'Ta181', 'Pb208', 'Th232'], dtype=object)

### raw sample names including sample sites and artefacts

In [50]:
my_data['Site'].unique()

array(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB',
       'BX', 'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB', 'AB',
       'LV', 'BR', 'KY', 'BF', 'ST', 'SH', 'CF', 'BG', 'AC', 'CR', 'GH',
       'PX', 'WF', 'DH', 'NMAG_Gold', 'NMW_Gold', 'NMWGwern', 'UBSS',
       'Cefn', 'Stockley', 'Pucha', 'Woodbury', 'Pimple', 'Wellington',
       'Lyonshall', 'SymondsYatE', 'Madawg', nan], dtype=object)

### define functions for making target classes for classification

In [51]:

def make_classes_grouped(row):
    if row['Geology'] == 'Bedrock':
        if row['Site'] == 'WB' or row['Site'] == 'BX':
            return('WB_BX')
        elif row['Site'] == 'BC' or row['Site'] == 'CS':
            return('BC_CS')
        elif row['Site'] == 'SQ' or row['Site'] == 'BP':
            return('SQ_BP')
        else:
            return(row['Site'])
    elif row['Geology'] == 'Superficial':
        if row['Region'] == 'SV' or row['Region'] == 'SE':
            return('SV_SE')
        else:
            return(row['Region'])
        
def make_classes_grouped_reduced(row):
    if row['Geology'] == 'Bedrock':
        if row['Site'] == 'WB' or row['Site'] == 'BX':
            return('WB_BX')
        else:
            return(row['Site'])
    elif row['Geology'] == 'Superficial':
        if row['Region'] == 'SV' or row['Region'] == 'SE':
            return('SV_SE')
        else:
            return(row['Region'])

def make_classes_raw(row):
    if row['Geology'] == 'Bedrock':
        return(row['Site'])
    elif row['Geology'] == 'Superficial':
        return(row['Region'])



     

### targets for classification are made

In [65]:
my_data['class'] = 'init'   

if grouping:
    my_data['class'] = my_data.apply(make_classes_grouped, axis = 1)
elif grouping_reduced:
    my_data['class'] = my_data.apply(make_classes_grouped_reduced, axis = 1)
elif raw:
    my_data['class'] = my_data.apply(make_classes_raw, axis = 1)

    


In [53]:
if drop_semi_bedrock:
    my_data = my_data[(my_data['class'] != 'BM') & (my_data['class'] != 'BC') & (my_data['class'] != 'BP') ]

### remove '<' signs and commas from feature values

In [54]:
for column_name in my_data.columns.values[9:-1]:
    def fill_less_than(row):
        if 'DL' in  str(row[column_name]):
            return(np.nan)
        if '<' in str(row[column_name]):
            return(float(row[column_name].replace('<', '').replace(',', '')))
        else:
            return(float(row[column_name]))
    my_data[column_name] = my_data.swifter.apply(fill_less_than, axis = 1)

Pandas Apply: 100%|██████████| 1627/1627 [00:00<00:00, 14416.46it/s]


### remove rows where there are all element abundances are na values 

In [75]:
my_data = my_data.dropna(subset=my_data.columns.values[9:-1], how = 'all' , axis = 0)

### Impute na values with feature mean

In [80]:
for column_name in my_data.columns.values[9:-1]:
    my_data[column_name] = my_data[column_name].fillna(my_data[column_name].mean()) 

### Outliers defined as any values that exceed 2 standard deviations from the mean, such values are changed to the mean for that variable

In [81]:
std_dict = {}
mean_dict = {}
median_dict = {}

for col in my_data.columns.values[9:-1]:
    std_dict[col] = my_data[col].std()
    
for col in my_data.columns.values[9:-1]:
    mean_dict[col] = my_data[col].mean()
    
for col in my_data.columns.values[9:-1]:
    median_dict[col] = my_data[col].median()
    

In [82]:
for col_name in my_data.columns.values[9:-1]:
    def impute_outliers(row):
        if np.abs(row[col_name] - mean_dict[col_name]) > 2*(std_dict[col_name]):
            return(mean_dict[col_name])
        else:
            return(row[col_name])
    my_data[col_name]= my_data.swifter.apply(impute_outliers, axis = 1)

Pandas Apply: 100%|██████████| 1596/1596 [00:00<00:00, 17016.38it/s]


### split data into 'train_data' and 'test_data', the former consists of samples from known geological sites and the latter from flint artefacts fow which the original geological source site is unknown and to be predicted.

In [60]:
my_data.isna().any()

Analysis     False
Geology      False
Province      True
Region        True
Site         False
SubSite       True
Formation     True
Band          True
Nodule       False
Li7          False
Be9          False
B11          False
Mg24         False
Al27         False
Si28         False
P31          False
S33          False
K39          False
Ca42         False
Sc45         False
Ti47         False
V51          False
Cr52         False
Mn55         False
Fe56         False
Co59         False
Ni60         False
Cu63         False
Zn68         False
Ga69         False
Ge72         False
As75         False
Rb85         False
Sr88         False
Y89          False
Zr90         False
Nb93         False
Mo95         False
Cd111        False
In115        False
Sn118        False
Cs133        False
Ba137        False
La139        False
Ce140        False
Pr141        False
Nd146        False
Sm147        False
Eu153        False
Gd157        False
Tb159        False
Dy163        False
Ho165       

In [83]:
train_data = my_data[(my_data['Geology']== 'Bedrock') | (my_data['Geology'] == 'Superficial')]
test_data = my_data[my_data['Geology']=='Artefacts']


### label encode the class to be predicted

### 'training_data' is split into two datasets, one consisting of samples from superficial sites and one containing samples from bedrock sites.

In [16]:
train_data_bedrock = train_data[train_data['Geology'] == 'Bedrock']
train_data_superficial = train_data[train_data['Geology'] == 'Superficial']

### four datasets are created, one containing all train data (bedrock and superficial types), one containing just bedrock types, one containing just superficial types and one containing the artefacts

In [17]:
element_data_train = train_data[train_data.columns.values[9:-1]]
element_data_train_bedrock = train_data_bedrock[train_data.columns.values[9:-1]]
element_data_train_superficial = train_data_superficial[train_data.columns.values[9:-1]]
element_data_test = test_data[test_data.columns.values[9:-1]]
element_data_everything = my_data[my_data.columns.values[9:-1]]

In [18]:
element_data_everything.head()

,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238
0,15.63,0.12,48.36,33.359881,943.71,464944.180000,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05
1,11.50,0.09,44.77,33.359881,1077.11,465010.940000,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04
2,20.05,0.06,44.88,42.700000,620.21,465295.410000,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05
3,11.16,0.73,47.06,33.359881,1143.19,462284.527976,2309.325714,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03
4,17.71,0.32,48.26,33.520000,547.22,465027.110000,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09


### features are standardised

In [19]:
my_scaler_train = StandardScaler()
my_scaler_train_bedrock = StandardScaler()
my_scaler_train_superficial = StandardScaler()
my_scaler_test = StandardScaler()
my_scaler_everything = StandardScaler()

In [20]:
element_data_train.head()

,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238
0,15.63,0.12,48.36,33.359881,943.71,464944.180000,50.280000,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05
1,11.50,0.09,44.77,33.359881,1077.11,465010.940000,70.910000,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04
2,20.05,0.06,44.88,42.700000,620.21,465295.410000,104.470000,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05
3,11.16,0.73,47.06,33.359881,1143.19,462284.527976,2309.325714,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03
4,17.71,0.32,48.26,33.520000,547.22,465027.110000,44.440000,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09


In [21]:
element_data_train_scaled = my_scaler_train.fit_transform(element_data_train)
element_data_train_bedrock_scaled = my_scaler_train_bedrock.fit_transform(element_data_train_bedrock)
element_data_train_superficial_scaled = my_scaler_train_superficial.fit_transform(element_data_train_superficial)
element_data_test_scaled = my_scaler_test.fit_transform(element_data_test)
element_data_everything_scaled = my_scaler_everything.fit_transform(element_data_everything)

### The four datasets are transformed using Principal component analysis 

In [22]:
my_pca_train = PCA(n_components=element_data_train_scaled.shape[1])
my_pca_train_bedrock = PCA(n_components=element_data_train_bedrock_scaled.shape[1])
my_pca_train_superficial = PCA(n_components=element_data_train_superficial_scaled.shape[1])
my_pca_test = PCA(n_components=element_data_test_scaled.shape[1])

element_data_train_pca = my_pca_train.fit_transform(element_data_train_scaled)
element_data_train_bedrock_pca = my_pca_train_bedrock.fit_transform(element_data_train_bedrock_scaled)
element_data_train_superficial_pca = my_pca_train_superficial.fit_transform(element_data_train_superficial_scaled)
element_data_test_pca = my_pca_test.fit_transform(element_data_test_scaled)

### print details of PCA

In [23]:
print(my_pca_train)

PCA(copy=True, iterated_power='auto', n_components=53, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


### the principal components for the four datasets are put into dataframes

In [24]:
no_PCs = element_data_train_scaled.shape[1]
PC_names = []
for i in range(0, no_PCs):
    number = i + 1
    column_name = 'PC' + str(number)
    PC_names.append(column_name)

In [25]:
PC_df_train = pd.DataFrame(data = element_data_train_pca, columns = PC_names)
PC_df_bedrock_train = pd.DataFrame(data = element_data_train_bedrock_pca, columns = PC_names)
PC_df_superficial_train = pd.DataFrame(data = element_data_train_superficial_pca, columns = PC_names)
PC_df_test = pd.DataFrame(data = element_data_test_pca, columns = PC_names)

### T-Distributed Stochastic Neighbour Embedding is done on the four datasets

In [62]:
#my_tsne_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_scaled)
#my_tsne_bedrock_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_bedrock_scaled)
#my_tsne_superficial_train = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_train_superficial_scaled)
#my_tsne_test = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_test_scaled)
my_tsne_everything = TSNE(n_components=3, n_iter=10000, verbose=3).fit_transform(element_data_everything_scaled)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1233 samples in 0.003s...
[t-SNE] Computed neighbors for 1233 samples in 0.296s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1233
[t-SNE] Computed conditional probabilities for sample 1233 / 1233
[t-SNE] Mean sigma: 1.999056
[t-SNE] Computed conditional probabilities in 0.104s
[t-SNE] Iteration 50: error = 72.4701614, gradient norm = 0.1052718 (50 iterations in 6.041s)
[t-SNE] Iteration 100: error = 73.2147751, gradient norm = 0.0825396 (50 iterations in 4.158s)
[t-SNE] Iteration 150: error = 74.2073975, gradient norm = 0.0817590 (50 iterations in 3.505s)
[t-SNE] Iteration 200: error = 74.3832169, gradient norm = 0.0830234 (50 iterations in 4.795s)
[t-SNE] Iteration 250: error = 74.2842026, gradient norm = 0.0741409 (50 iterations in 4.660s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 74.284203
[t-SNE] Iteration 300: error = 1.5742738, gradient norm = 0.0006406 (50 iterations in 5.797s)

### the t-SNE dimensions for the four datasets are put into dataframes

In [32]:
#tsne_df_train = pd.DataFrame(data = my_tsne_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_bedrock_train = pd.DataFrame(data = my_tsne_bedrock_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_superficial_train = pd.DataFrame(data = my_tsne_superficial_train, columns = ['tsne1', 'tsne2', 'tsne3'])
#tsne_df_test = pd.DataFrame(data = my_tsne_test, columns = ['tsne1', 'tsne2', 'tsne3'])
tsne_df_everything = pd.DataFrame(data = my_tsne_everything, columns = ['tsne1', 'tsne2', 'tsne3'])

In [33]:
bedrock_data = train_data[train_data['Geology'] == 'Bedrock']
superficial_data = train_data[train_data['Geology'] == 'Superficial'].reset_index(drop = True)
artefact_data = test_data.reset_index(drop = True)

In [34]:
def addartefactclass(row):
    if row['Geology'] == 'Bedrock':
        return(row['class'])
    elif row['Geology'] == 'Superficial':
        return(row['class'])
    else:
        return('Artefact')
my_data['class'] = my_data.swifter.apply(addartefactclass, axis = 1)

Pandas Apply: 100%|██████████| 1596/1596 [00:00<00:00, 18098.74it/s]


In [36]:
#tsne_df_train_labelled = pd.concat([tsne_df_train, train_data['class']], axis = 1)
#tsne_df_bedrock_train_labelled = pd.concat([tsne_df_bedrock_train, bedrock_data['class']], axis = 1)
#tsne_df_superficial_train_labelled = pd.concat([tsne_df_superficial_train, superficial_data['class']], axis = 1)
#tsne_df_test['class'] = 1
tsne_df_everything_labelled = pd.concat([tsne_df_everything, my_data['class']], axis = 1)

In [45]:
tsne_df_everything_labelled

,tsne1,tsne2,tsne3,class
0,6.121238,-4.644475,-37.981857,FH
1,5.614727,-6.245814,-35.649292,FH
2,9.089909,-4.486228,-40.054489,FH
3,-5.881431,29.455050,-8.911158,FH
4,10.037048,-4.630585,-38.265583,FH
5,4.472555,1.268591,-41.633755,FH
6,4.606206,-5.034371,-32.763081,FH
7,7.324140,-1.928022,-39.590248,FH
8,2.465045,-0.744769,-40.821609,FH
9,7.201707,1.975347,-42.000088,FH


In [44]:
my_data['class'].unique()

array(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG',
       'WB_BX', 'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB',
       'AB', 'LV', 'SV_SE', 'BA', 'WA', 'MM', 'Artefact'], dtype=object)

In [81]:
tsne_df_everything.index

RangeIndex(start=0, stop=1233, step=1)

In [104]:
element_data_everything.shape

(1233, 53)

In [40]:
my_data.shape

(1596, 63)

In [69]:
tes = pd.concat([tsne_df_everything,my_data['class']], axis = 1)

In [31]:
tsne_df_everything_labelled.tail()

NameError: name 'tsne_df_everything_labelled' is not defined

### Do same as above cell but for PCA

In [ ]:
tsne_df_superficial_train_labelled.head()

In [60]:
#tsne_df_train_labelled.to_csv('tsne_both.csv')
#tsne_df_bedrock_train_labelled.to_csv('tsne_bedrock.csv')
#tsne_df_superficial_train_labelled.to_csv('tsne_superficial.csv')
#tsne_df_test.to_csv('tsne_artefacts.csv')
#tsne_df_everything_labelled.to_csv('tsne_samples_and_artefacts.csv')